# Network-based Kmeans clustering

The example below demonstrates network-based clustering using optimal transport Kmeans algorithm and survival analysis using Kaplan-Meier analysis.

This code reads three files: (1) partial correlations among features, (2) preprocessed radiomic features, and (3) clinical data including survival status and follow-up time.

**Citation:** Oh et al, SU-400-360, AAPM 2023 and Oh et al, Journal of Medical Imaging, 8(3):031904, 2021

In [ ]:
EBICgraph_file = "EBICgraph.csv"
preprocessed_radiomics_file = "preprocessed_data_red.csv"
clinical_file = "clinical_data.xlsx"

## Install OMT and CVX packages

In [ ]:
!pip install pot
!pip install cvxpy

## Install other packages

In [ ]:
!pip install seaborn==0.11.2
!pip install openpyxl
!pip install lifelines
!pip install bioinfokit

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import numpy.matlib
%matplotlib inline

import math
from sklearn import set_config
from sklearn.impute import SimpleImputer

import networkx as nx
from networkx.algorithms import dfs_labeled_edges

import ot
import cvxpy as cp

set_config(display="text")

## Import the unblanced OMT library

In [ ]:
from UnbalancedOMT import UnbalancedOMT

## Read radiomic data

In [ ]:
# Upload your own data
df_all = pd.read_csv(preprocessed_radiomics_file)
df_red = df_all.iloc[:,1:len(df_all.columns)]

In [ ]:
df_all

## Read adjacency matrix

In [ ]:
# Upload adjacency data
corr_data = pd.read_csv(EBICgraph_file)
corr_data = corr_data.iloc[:,1:len(corr_data.columns)]

df_corr = corr_data.copy()
df_corr=df_corr.set_index(df_red.columns)
print(df_corr)

## Draw the entire network

In [ ]:
cor_matrix = np.asmatrix(df_corr)
n=len(df_red.columns)
variables=df_red.columns
print(variables)
threshold=0.15 # Threshold of partial correlation
G=nx.Graph()

for i in range(n):
    for j in range(n):
      if i> j and abs(cor_matrix[i,j]) > threshold:
        G.add_edge(variables[i],variables[j], weight=1)

plt.figure(1,figsize=(9,9))
nx.draw(G, pos = nx.nx_pydot.graphviz_layout(G), node_size=20, \
    node_color='lightblue', linewidths=0.25, font_size=0.1, \
    font_weight='bold', with_labels=True )
adj = nx.adjacency_matrix(G)

# Find a largest network component

In [ ]:
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)

# Choose the network (island) of interest
ISLAND = 0  # 0: the largest connected network, 1: the second largest one, and so on
G0 = G.subgraph(Gcc[ISLAND])
plt.figure(2,figsize=(20,20))
nx.draw(G0, pos = nx.nx_pydot.graphviz_layout(G0), node_size=100, \
    node_color='lightblue', linewidths=0.25, font_size=8, \
    font_weight='bold', with_labels=True )
adj_G0 = nx.adjacency_matrix(G0)

nodes0=G0.nodes # the largest one
df_G0=df_red[list(nodes0)]
print(adj_G0)

## Print nodes

In [ ]:
print(list(nodes0))

['Gabor_voxSz111mm_Sigma4mm_AR1_5_wavLen8mm_OrientAvg_30609012015_firstOrderS_std', 'Gabor_voxSz111mm_Sigma4mm_AR1_5_wavLen8mm_OrientAvg_30609012015_firstOrderS_P90', 'Gabor_voxSz111mm_Sigma4mm_AR1_5_wavLen8mm_OrientAvg_30609012015_firstOrderS_medianAbsDev', 'LawsEnergy_3d_typeE5E5E5_normyes_energyKernelSize3xx3xx3_firstOrderS_P10', 'LawsEnergy_3d_typeS5S5S5_normyes_energyKernelSize3xx3xx3_firstOrderS_kurtosis', 'Gabor_voxSz111mm_Sigma4mm_AR1_5_wavLen8mm_OrientAvg_30609012015_firstOrderS_robustMedianAbsDev', 'LawsEnergy_3d_typeE5E5E5_normyes_energyKernelSize3xx3xx3_firstOrderS_range', 'LawsEnergy_3d_typeS5S5S5_normyes_energyKernelSize3xx3xx3_firstOrderS_robustMedianAbsDev', 'LawsEnergy_3d_typeE5E5E5_normyes_energyKernelSize3xx3xx3_firstOrderS_coeffDispersion', 'LawsEnergy_3d_typeE5E5E5_normyes_energyKernelSize3xx3xx3_firstOrderS_max', 'LawsEnergy_3d_typeE5E5E5_normyes_energyKernelSize3xx3xx3_firstOrderS_median', 'Gabor_voxSz111mm_Sigma4mm_AR1_5_wavLen8mm_OrientAvg_30609012015_firstOrde

# Balanced OMT

In [ ]:
def dist_cvx(drho,D,m):

    # Create two scalar optimization variables.
    u = cp.Variable((m))

    # Form the objective function.
    objective = cp.Minimize(cp.sum(cp.abs(u)))

    # Create two constraints.
    constraints = [drho-D@u == 0]

    # Solve the problem.
    problem = cp.Problem(objective, constraints)
    problem.solve()
    return problem.value

# Functions for Wasserstein distance-based Kmeans

In [ ]:
def initiate_centroids(k, dset):
    '''
    Select k data points as centroids
    k: number of centroids
    dset: pandas dataframe
    '''
    centroids = dset.sample(k)
    return centroids

def rsserr(a,b):
    return np.square(np.sum((a-b)**2))

def centroid_assignation(G0,gamma,UNBALANCED, dset, centroids, D, m, no_feature):
    k = centroids.shape[0]
    n = dset.shape[0]
    assignation = []
    assign_errors = []

    for obs in range(n):
        # Estimate error
        all_errors = np.array([])
        for centroid in range(k):

            if UNBALANCED==1:
              rho0=centroids.iloc[centroid, 0:no_feature].to_numpy()
              rho1=dset.iloc[obs,0:no_feature].to_numpy()

              b=UnbalancedOMT(rho0,rho1,G0,gamma)
              err=b.UnbalancedDistance()
            else:
              #WD
              X2=np.stack((centroids.iloc[centroid, 0:no_feature].to_numpy(), dset.iloc[obs,0:no_feature].to_numpy()))
              X2=np.transpose(X2)

              Node_weights = X2
              S = X2.sum(axis=0)
              S_copy = np.matlib.repmat(S,no_feature,1)
              Node_Prob = Node_weights/S_copy

              rho0 = Node_Prob[:,0]
              rho1 = Node_Prob[:,1]
              drho = rho0-rho1
              drho = drho-np.average(drho)
              err = dist_cvx(drho,D,m)

            all_errors = np.append(all_errors, err)

        # Get the nearest centroid and the error
        nearest_centroid =  np.where(all_errors==np.amin(all_errors))[0].tolist()[0]
        nearest_centroid_error = np.amin(all_errors)

        # Add values to corresponding lists
        assignation.append(nearest_centroid)
        assign_errors.append(nearest_centroid_error)

    return assignation, assign_errors

## Wasserstein distance-based Kmeans using unbalanced OMT

In [ ]:
def kmeans(G0,gamma,UNBALANCED, dset, k, D, m, no_feature, dis, tol=1e-5):
    '''
    K-means implementationd for a
    `dset`:  DataFrame with observations
    `k`: number of clusters, default k=2
    `tol`: tolerance=1E-4
    '''

    working_dset = dset.copy()
    err = []
    goahead = True
    j = 0

    # Initiate clusters by defining centroids
    centroids = initiate_centroids(k, dset)
    ITER=5
    while(goahead):
        print(j)
        # Assign centroids and calculate error
        working_dset['centroid'], j_err = centroid_assignation(G0,gamma,UNBALANCED,working_dset, centroids, D, m, no_feature)
        err.append(sum(j_err))

        #  Update centroid position
        centroids = working_dset.groupby('centroid').agg('mean').reset_index(drop = True)

        # Restart the iteration
        if j>0:
            if np.abs(err[j-1]-err[j])<=tol:
                goahead = False
        j+=1

    working_dset['centroid'], j_err = centroid_assignation(G0,gamma,UNBALANCED,working_dset, centroids, D, m, no_feature)
    centroids = working_dset.groupby('centroid').agg('mean').reset_index(drop = True)

    return working_dset['centroid'], j_err, centroids

## Initialization for balanced OMT

In [ ]:
T = df_G0.to_numpy()
T = np.transpose(T)
no_sample = T.shape[1]
no_feature = T.shape[0]
print(no_feature)

m = int(round(adj_G0.sum()/2))
print(adj_G0.sum())
print(m)
D = np.zeros((no_feature, m))
count = 0

for i in range(no_feature-1):
    for j in range(i+1,no_feature):
        if adj_G0[i,j]==1:
            D[i,count] = -1
            D[j,count] = 1
            count=count+1

## Run Wasserstein Kmeans

In [ ]:
UNBALANCED = 1
NO_CLUSTER = 2 # number of clusters
gamma = 30 # paramenter for unblanced OMT
G_0_dis=nx.floyd_warshall_numpy(G0)
df_G0_copy=df_G0.copy()

mapping = dict(zip(G0, range(0, df_G0_copy.shape[1])))
G0 = nx.relabel_nodes(G0, mapping)

df_G0['centroid'], df_G0['error'], centroids =  kmeans(G0,gamma,UNBALANCED,df_G0, NO_CLUSTER, D, m, no_feature, G_0_dis)

## TSNE visualization

In [ ]:
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2, learning_rate='auto',
                   init='random').fit_transform(df_G0.iloc[:,:-2])


In [ ]:
centroid= df_G0['centroid'].tolist()


sns.set(rc={'figure.figsize':(8,6)})

sns.set(font_scale = 2)
sns.set_style("whitegrid")
sns.scatterplot(X_embedded[:,0], X_embedded[:,1], hue=centroid)

plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.legend(['Class 1', 'Class 2'])

ax = plt.gca()
leg = ax.get_legend()
leg.legendHandles[0].set_color('darkblue')
leg.legendHandles[1].set_color('darkorange')

## Read clinical data

In [ ]:
# Upload clinical data including survival status and follow-up time
df_cli = pd.read_excel(clinical_file)
df_all_ids=df_all['id']

df_cli_matched=df_cli.loc[df_cli['Case ID'].isin(df_all_ids)]

## Survival analysis

In [ ]:
# KM analysis for two clusters
from lifelines.statistics import logrank_test
from lifelines.statistics import multivariate_logrank_test
from matplotlib.offsetbox import AnchoredText
from lifelines import KaplanMeierFitter, CoxPHFitter

OS=df_cli_matched['Survival'].to_numpy()
OS_time=df_cli_matched['Survival time'].to_numpy()

centroid= df_G0['centroid']
idx0= np.where(centroid == 0)
idx1= np.where(centroid == 1)

kmf = KaplanMeierFitter()
ax = plt.subplot(111)
h_OS_time=OS_time[idx0]
h_OS=OS[idx0]
kmf.fit(h_OS_time/365, event_observed = h_OS, label = 'Low risk')
kmf.plot(ax = ax, show_censors=True)

m_OS_time=OS_time[idx1]
m_OS=OS[idx1]
kmf.fit(m_OS_time/365, event_observed = m_OS, label = 'High risk')
kmf.plot(ax = ax, show_censors=True)

#plt.title("Kaplan Meier estimates in lung cancer")
plt.xlabel("Years")
plt.ylabel("Survival")
plt.rcParams["figure.figsize"] = (13,8)
plt.rcParams.update({'font.size': 22})

result = multivariate_logrank_test(OS_time, centroid, OS)
result.test_statistic
result.p_value
result.print_summary()